In [ ]:
import pandas as pd
import datetime
import requests
import json
import math
import numpy as np

## key 저장

In [ ]:
f = open("keyfile.txt", 'r')
key = {}
for i in range(1, 11):
    key[i] = f.readline().rstrip()
    print('key'+str(i))
f.close()

In [ ]:
key

In [ ]:
key[1]

## 데이터 목록
### 1. 영화 목록
* INPUT
    - 키 값, 현재 페이지, 결과 ROW 의 개수, 영화명, 감독명, 조회시작 개봉연도(YYYY), 조회종료 개봉연도(YYYY), 조회시작 제작연도(YYYY), 조회종료 제작연도(YYYY), 국적코드, 영화유형코드

* OUTPUT
    - 영화코드, 영화명(국문),영화명(영문), 제작연도, 개봉연도, 영화유형, 제작상태, 제작국가(전체), 영화장르(전체), 대표 제작국가명, 대표 장르명, 영화감독, 영화감독명, 제작사, 제작사 코드, 제작사명

### 2. 영화 상세정보
* INPUT
    - 키 값, 영화코드
    
* OUTPUT
    - 영화코드, 영화명(국문), 영화명(영문), 영화명(원문), 제작연도, 상영시간, 개봉연도, 제작상태명, 영화유형명, 제작국가, 제작국가명, 장르명, 감독, 감독명, 감독명(영문), 배우, 배우명, 배우명(영문), 배역명, 배역명(영문), 상영형태 구분1, 상영형태 구분2, 상영형태명, 심의정보, 심의번호, 관람등급 명칭, 참여 영화사, 참여 영화사 코드, 참여 영화사명, 참여 영화사명(영문), 참여 영화사 분야명, 스텝, 스텝명, 스텝명(영문), 스텝역할명
    
### 3. 영화사 목록
* INPUT
    - 키 값, 현재 페이지(default : “1”), 결과 ROW 의 개수(default : “10”), 영화사명, 대표자명, 분류코드(default: 전체)
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 영화사 분류, 대표자명, 필모리스트

### 4. 영화사 상세정보
* INPUT
    - 키 값, 영화사코드
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 대표자명, 영화사 분류, 영화사 분류명, 영화사 필모, 참여 영화코드, 참여 영화명, 영화사 참여 분류명

### 5. 영화인 목록
* INPUT
    - 키 값, 현재 페이지를 지정(default : “1”), 결과 ROW 의 개수(default : “10”), 영화인명, 필모리스트
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 분야, 필모리스트

### 6. 영화인 상세정보
* INPUT
    - 키 값, 영화인코드
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 성별, 영화인 분류명, 영화인 필모, 참여 영화코드, 참여 영화명, 참여 분야, 관련 URL


## 1. 영화 리스트들 중에서 2019 박스오피스로 필터링 된 영화들 정보 가져오기(xlsx)

상영 횟수 40회 이하 아웃라이어 제거 위함(영진위 한국영화성평등소위원회 분석 참고). 


In [ ]:
movies_2019_filtered = pd.read_excel('KOBIS_boxoffice_2019_filtered.xlsx')
movies_2019_filtered = movies_2019_filtered[:-1]
movies_2019_filtered

In [ ]:
movies_2019_filtered.isnull().sum()

In [ ]:
movies_2019_filtered.fillna('-', inplace=True)

In [ ]:
movies_2019_filtered[movies_2019_filtered['감독 '] == '-']

### 영화 기본정보 가져오는 함수

In [ ]:
# 페이지 수, 조회시작 개봉연도, 조회종료 개봉연도를 파라미터(입력값)로 지정해준다.
def get_movies(movieNm, year, directorNm):
    # 영화 목록 API JSON URL
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json'
    
    # 입력받은 영화이름, 년도, 감독이름으로 API 호출.
    if directorNm == '-':
        payload = {
            'key': key[1]
            , 'itemPerPage': '100' # 한페이지에 한번에 불러올 수 있는 최대 데이터의 갯수
            , 'movieNm':movieNm
            , 'openStartDt': year
        }
    else:
         payload = {
            'key': key[1]
            , 'itemPerPage': '100' # 한페이지에 한번에 불러올 수 있는 최대 데이터의 갯수
            , 'movieNm':movieNm
            , 'directorNm':directorNm
        }

    # 결과를 변수에 담고 json 형식으로 변환해준다.
    res = requests.get(url, params=payload).json()
    
    # 쓸데 없는 겉부분 버리고 영화 리스트 컨텐츠만 가져옴
    return res['movieListResult']['movieList']

In [ ]:
# 함수 테스트 감독명 없을 경우
res = get_movies('극한직업', 2019, '-')

In [ ]:
pd.DataFrame(res)

### 영화 기본정보 가져오는 함수 돌려서 영화 코드 가져오기

In [ ]:
# 영화 몇개나 있는지 확인
len(movies_2019_filtered)

In [ ]:
# 컬럼 이름 틀리지 않도록 확인. *'감독 '* 주의
movies_2019_filtered.columns

In [ ]:
# 함수에 필요한 정보(영화명, 감독명, 개봉일) 가져오기
movieNm_list = movies_2019_filtered['영화명']
movieNm_list

In [ ]:
# 감독명은 한명만 있어도 검색 가능
directorNm_list = movies_2019_filtered['감독 '].apply(lambda x: x.split(',')[0])
directorNm_list

In [ ]:
# 개봉일은 년도로만 검색 가능
openYear_list = movies_2019_filtered['개봉일'].apply(lambda x: x.year)
openYear_list

In [ ]:
movie_info_list = []

for movie in zip(movieNm_list, openYear_list, directorNm_list):
    print(movie)
    movie_info_list.append(get_movies(movie[0], movie[1], movie[2])[0])

In [ ]:
movie_baisc_info = pd.DataFrame(movie_info_list)
movie_baisc_info

<br>\
<br>
### 영화 상세정보 함수

영화코드를 가져왔으니 이제 영화 상세정보를 가져올 수 있습니다.

In [ ]:
def get_movie_details(movieCd):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'

    payload = {
        'key': key[1]
        , 'movieCd': movieCd
    }

    res = requests.get(url, params=payload).json()
    
    return res['movieInfoResult']['movieInfo']

영화 기본 정보에서 알 수 있는 정보는 제외, 이외에 상세정보에서만 얻을 수 있는 데이터들 확인.

1. 러닝타임
2. 심의 정보
3. 감독 정보
4. 배우 정보
5. 제작사 정보
6. 스태프 정보

In [ ]:
# get_movie_details 함수 테스트
res = get_movie_details(20182530)
res

### 후가공을 위한 데이터 확인

영화 상세정보 호출 결과 그대로 데이터프레임화 할 수 없으니 필요한 정보들을 종류별로 가져와서 따로따로 저장해서 테이블화 해야겠다.

1. 러닝타임

In [ ]:
res['showTm']

2. 심의 정보

In [ ]:
res['audits']

3. 감독 정보
4. 배우 정보
5. 제작사 정보
6. 스태프 정보

In [ ]:
res['directors']

In [ ]:
res['actors']

In [ ]:
res['companys']

In [ ]:
res['staffs']

### 후가공

1. 상세정보를 확인하고 싶은 영화리스트에서 영화 코드만 뽑아옵니다.
2. for문 돌면서 get_movie_details로 API 호출.
3. 각각의 영화에 대한 API 호출 결과를 받아서 필요한 정보들을 데이터프레임화 할 수 있는 형태로 리스트에 각각 저장
4. 필요한 경우에 영화 코드로 조인해서 하나의 데이터로 사용할 수 있게 각각의 데이터와 영화코드 함께 딕셔너리 형태로 저장

In [ ]:
# 영화 코드만 뽑아오기
movieCd_list = movie_baisc_info['movieCd']

In [ ]:
movie_showTm = []
movie_audits = []
movie_directors = []
movie_actors = []
movie_staffs = []
movie_companys = []

# 테스트 할 때는 `movieCd_list.iloc[:1]` 하나만 돌려보세요.

# movieCd_list 길이가 3000이 넘어갈 경우 for문을 쪼개서 돌려야 합니다. 함수 정의된 부분으로 가서 key도 바꿔줘야 해요!.
# ex) movieCd_list.iloc[:3000] 이런 식으로 구간을 나눠서 여러번 함수를 실행시켜야겠죠?
for code in movieCd_list:
    res = get_movie_details(code)
    movieNm = res['movieNm']
    print(code)
    

    showTm_dict = {}
    showTm_dict['movieCd'] = code
    showTm_dict['showTm'] = res['showTm']
    movie_showTm.append(showTm_dict)

    for audit in res['audits']:
        audits_dict = {}
        audits_dict['movieCd'] = code
        audits_dict['auditNo'] = audit['auditNo']
        audits_dict['watchGradeNm'] = audit['watchGradeNm']
        movie_audits.append(audits_dict)
    
    for director in res['directors']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = director['peopleNm']
        person_info['repRoleNm'] = '감독'
        movie_directors.append(person_info)
    for actor in res['actors']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = actor['peopleNm']
        person_info['repRoleNm'] = '배우'
        person_info['cast'] = actor['cast']
        movie_actors.append(person_info)
    for staff in res['staffs']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = staff['peopleNm']
        person_info['staffRoleNm'] = staff['staffRoleNm']
        movie_staffs.append(person_info)
    
    for company in res['companys']:
        person_info = {}
        person_info['movieCd'] = code
        person_info['companyCd'] = company['companyCd']
        person_info['companyNm'] = company['companyNm']
        person_info['companyPartNm'] = company['companyPartNm']
        movie_companys.append(person_info)

추려진 데이터들 확인

In [ ]:
showTm_df = pd.DataFrame(movie_showTm)
showTm_df.to_csv('showTm_2019_ko.csv', index=False)

In [ ]:
audits_df = pd.DataFrame(movie_audits)
audits_df.to_csv('audits_2019_ko.csv', index=False)

In [ ]:
directors_df = pd.DataFrame(movie_directors)
directors_df.to_csv('directors_2019_ko.csv', index=False)

In [ ]:
actors_df = pd.DataFrame(movie_actors)
actors_df.to_csv('actors_2019_ko.csv', index=False)

In [ ]:
staffs_df = pd.DataFrame(movie_staffs)
staffs_df.to_csv('staffs_2019_ko.csv', index=False)

In [ ]:
companys_df = pd.DataFrame(movie_companys)
companys_df.to_csv('companys_2019_ko.csv', index=False)

### # 주의 # 한 테스크가 끝날 때마다 csv로 저장하고 다음 작업 시작할 때는 저장된 csv 불러와서 시작해야 합니다.

In [ ]:
directors_df = pd.read_csv('directors_2019_ko.csv')

In [ ]:
actors_df = pd.read_csv('actors_2019_ko.csv')

In [ ]:
staffs_df = pd.read_csv('staffs_2019_ko.csv')

In [ ]:
companys_df = pd.read_csv('companys_2019_ko.csv')

<br>\
<br>\
<br>

### 잘 돌아가다가 KeyError: 'peopleInfoResult'가 발생했을 때.

해당에러가 뜰 경우에는 key 사용 한도를 초과한 것입니다.

멈추기 전까지는 데이터가 다 저장되어있으니 당황하지 마시고 어디까지 저장되었는지 아래처럼 범위를 이용해서 또는 .tail() 함수를 이용해 데이터를 확인하고 다시 시작해야하는 부분이 어디인지 알아냅니다.

In [ ]:
# 데이터 확인
staff_detail_list_2[-10:]

In [ ]:
# '김효주', '아메리카 타운' 정보가 맨 마지막일 경우
# index값을 확인한다. 3000까지 뽑힌거니까 3001부터 데이터를 다시 가져오면 됨.
peopleCd_list[(peopleCd_list['peopleNm'] == '김효주') & (peopleCd_list['movieNm'] == '아메리카 타운')]